In [ ]:
!pip install geemap
!pip install geopandas
!pip install shapely

import ee
import geemap
import geopandas as gpd
from shapely.geometry import mapping

# Authenticate and initialize the Earth Engine library
ee.Authenticate()
ee.Initialize(project='ee-wa-lclu')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ee.Initialize(project = 'ee-wa-lclu')

In [ ]:
# Define the date range and select the bands.
start_date = '2017-01-01'
end_date = '2024-5-31'
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']

In [ ]:
# Provide the path to your shapefile
shapefile_path = '/content/drive/MyDrive/EO/boundary/wa_shp.shp'

import geopandas as gpd
from shapely.geometry import mapping
import json

# Provide the path to your shapefile
shapefile_path = '/content/drive/MyDrive/EO/boundary/wa_shp.shp'

# Read the shapefile
gdf = gpd.read_file(shapefile_path)

# Ensure the shapefile is in the correct projection (WGS84)
gdf = gdf.to_crs(epsg=4326)

# Convert the GeoDataFrame to a GeoJSON dictionary
geojson_dict = json.loads(gdf.to_json())

# Extract the geometry coordinates
polygon_coords = geojson_dict['features'][0]['geometry']['coordinates']

# Check the coordinates format and length
print(f"Geometry type: {gdf.geometry.type[0]}")
print(f"Number of polygons: {len(polygon_coords)}")
print(f"First polygon coordinates (truncated): {polygon_coords[0][:5]}")

Geometry type: Polygon
Number of polygons: 1
First polygon coordinates (truncated): [[-77.119795219, 38.934350904, 0.0], [-77.119794908, 38.9343511390001, 0.0], [-77.040966493, 38.9959680370001, 0.0], [-76.909149956, 38.892931436, 0.0], [-77.0390228859999, 38.7916443510001, 0.0]]


In [ ]:
print(polygon_coords)


[[[-77.119795219, 38.934350904, 0.0], [-77.119794908, 38.9343511390001, 0.0], [-77.040966493, 38.9959680370001, 0.0], [-76.909149956, 38.892931436, 0.0], [-77.0390228859999, 38.7916443510001, 0.0], [-77.039022636, 38.791882392, 0.0], [-77.039042782, 38.793213881, 0.0], [-77.039123568, 38.7968752570001, 0.0], [-77.0391441529999, 38.7976767910001, 0.0], [-77.039185317, 38.8005909670001, 0.0], [-77.0389955749999, 38.801626018, 0.0], [-77.0387846639999, 38.8027829070001, 0.0], [-77.038334768, 38.8051019700001, 0.0], [-77.037483277, 38.809405778, 0.0], [-77.037054846, 38.8109810680001, 0.0], [-77.0364945709999, 38.812986761, 0.0], [-77.036131459, 38.814280502, 0.0], [-77.036139996, 38.8148124950001, 0.0], [-77.038129097, 38.815596385, 0.0], [-77.038138202, 38.8156127770001, 0.0], [-77.0381474229999, 38.8156301600001, 0.0], [-77.038155838, 38.8156464630001, 0.0], [-77.0381642519999, 38.815662225, 0.0], [-77.0381723209999, 38.815679339, 0.0], [-77.0381810809999, 38.8156954610001, 0.0], [-77.0

In [ ]:
import ee

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

# Remove the Z coordinate (altitude) from the coordinates if present
def remove_z(coords):
    return [[coord[:2] for coord in poly] for poly in coords]

polygon_coords_2d = remove_z(polygon_coords)

# Create an Earth Engine Polygon object from the coordinates
try:
    roi = ee.Geometry.Polygon(polygon_coords_2d)
    print("Geometry successfully created!")
except Exception as e:
    print("Error creating geometry:", e)

Geometry successfully created!


In [ ]:
roi

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Define a function to mask clouds using the Sentinel-2 QA band
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
        qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

# Load Sentinel-2 data for 2017
s2_2017 = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2017-01-01', '2017-12-31') \
    .filterBounds(roi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'])

# Load Sentinel-2 data for 2024
s2_2024 = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2024-01-01', '2024-12-31') \
    .filterBounds(roi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'])



In [ ]:
# Median composite for 2017
composite_2017 = s2_2017.median().clip(roi)

# Median composite for 2024
composite_2024 = s2_2024.median().clip(roi)

In [ ]:
# Define export parameters
export_params = {
    'scale': 10,
    'region': roi,
    'driveFolder': 'EarthEngine',
    'fileFormat': 'GeoTIFF'
}

# Export 2017 composite
export_task_2017 = ee.batch.Export.image.toDrive(composite_2017, 'Sentinel2_2017', **export_params)
export_task_2017.start()

# Export 2024 composite
export_task_2024 = ee.batch.Export.image.toDrive(composite_2024, 'Sentinel2_2024', **export_params)
export_task_2024.start()